In [70]:
import numpy as np
from scipy import linalg as lg
import time

In [71]:
def count_layers(len_dim, factors):
    layers = []
    layer = []
    for n in range(len_dim):
        layer.append(factors[n].T @ factors[n])
    layers.append(layer)

    for n in range(1, len_dim - 1):
        layer = []
        for r in range(len_dim - n):
            layer.append(layers[n - 1][r] * layers[0][r + n])
        layers.append(layer)
    return layers

In [72]:
def count_W(len_dim, layers):
    W = []
    for row in range(len_dim):
        start_layer = row - 1
        end_layer = len_dim - row - 2
        if start_layer >= 0:
            W_col = layers[start_layer][0]
            if end_layer >= 0:
                W_col = W_col * layers[end_layer][-1]
        else:
            W_col = layers[end_layer][-1]

        W_row = [W_col]

        for col in range(row + 1, len_dim):
            start_layer = row - 1
            middle_layer = col - row - 2
            end_layer = len_dim - col - 2

            if start_layer >= 0:
                W_col = layers[start_layer][0]
                if middle_layer >= 0:
                    W_col = W_col * layers[middle_layer][row + 1]
                if end_layer >= 0:
                    W_col = W_col * layers[end_layer][-1]
            elif middle_layer >= 0:
                W_col = layers[middle_layer][row + 1]
                if end_layer >= 0:
                    W_col = W_col * layers[end_layer][-1]
            else:
                W_col = layers[end_layer][-1]


            W_row.append(W_col)

        W.append(W_row)

    return W

In [73]:
def count_JJ_d(len_dim, W, pos, jacob_size, dimensions, rank, factors):
    JJ = np.zeros((jacob_size, jacob_size))
    for row in range(len_dim):
        BB = W[row][0]
        for r in range(dimensions[row]):
            now_pos = pos[row] + r * rank
            JJ[now_pos:now_pos+rank, now_pos:now_pos+rank] = BB

    for alpha in range(len_dim):
        now_alpha = pos[alpha]
        for i in range(alpha + 1, len_dim):
            now_i = pos[i]

            now_W = W[alpha][i - alpha]
            for betta in range(dimensions[alpha]):
                now_a_b = now_alpha + betta * rank
                for j in range(dimensions[i]):
                    now_i_j = now_i + j * rank
                    for gamma in range(rank):
                        now_a_b_g = now_a_b + gamma
                        for k in range(rank):
                            now_i_j_k = now_i_j + k
                            JJ[now_a_b_g, now_i_j_k] = now_W[gamma][k] * factors[i][j][gamma] * factors[alpha][betta][k]
                            JJ[now_i_j_k, now_a_b_g] = now_W[k][gamma] * factors[i][j][gamma] * factors[alpha][betta][k]
    return JJ

In [74]:
def calculate_JA(len_dim, factors, rank, dimensions, W, jacob_size):
    JA = np.zeros(jacob_size)
    for i in range(len_dim):
        BB = W[i][0]
        pos_i = int((np.sum(dimensions[:i])) * rank)
        for j in range(dimensions[i]):
            pos_j = pos_i + j * rank
            for k in range(rank):
                pos_k = pos_j + k
                val = 0
                for r in range(rank):
                    val += BB[k, r] * factors[i][j, r]
                JA[pos_k] = val

    return JA

In [101]:
def calculate_JT(len_dim, factors, rank, dimensions, jacob_size, tensor):
    JT = np.zeros(jacob_size)

    prom = np.array(tensor.reshape(-1, dimensions[len_dim - 1]) @ factors[len_dim - 1][:, 0])

    for r in range(1, rank):
        prom = np.vstack((prom, np.array(tensor.reshape(-1, dimensions[len_dim - 1]) @ factors[len_dim - 1][:, r])))
    BT = [prom.T]

    for i in range(1, len_dim - 1):

        if rank !=1:
            prom = np.array(BT[i - 1][:, 0].reshape(-1, dimensions[len_dim - 1 - i]) @ factors[len_dim - 1 - i][:, 0])
        else:
            prom = np.array(BT[i - 1].reshape(-1, dimensions[len_dim - 1 - i]) @ factors[len_dim - 1 - i][:, 0])

        for r in range(1, rank):
            prom = np.vstack((prom, np.array(BT[i - 1][:, r].reshape(-1, dimensions[len_dim - 1 - i]) @ factors[len_dim - 1 - i][:, r])))

        BT.append(prom.T)

    for i in range(len_dim):

        if i != 0 and i != len_dim - 1:

            small_permute = np.array([i])
            small_permute = np.append(small_permute, np.arange(i))
            small_dim = np.array(dimensions[i])
            small_dim = np.append(small_dim, dimensions[:i])

            B_i = None
            for r in range(rank):

                if rank != 1:
                    prom = np.transpose(BT[len_dim - 2 - i][:, r].reshape(dimensions[:(i + 1)]), small_permute)
                else:
                    prom = np.transpose(BT[len_dim - 2 - i].reshape(dimensions[:(i + 1)]), small_permute)

                for j in range(i - 1, -1, -1):
                    prom = prom.reshape(-1, dimensions[j]) @ factors[j][:, r]
                    prom = prom.reshape(small_dim[:(j + 1)])
                    small_permute[0] -= 1
                if B_i is None:
                    B_i = prom
                else:
                    B_i = np.vstack((B_i, prom))
                small_permute[0] = i

            if rank != 1:
                JT[int(np.sum(dimensions[:i]) * rank) : int(np.sum(dimensions[:(i+1)]) * rank)] = np.transpose(B_i, (1, 0)).reshape((-1,))
            else:
                JT[int(np.sum(dimensions[:i]) * rank) : int(np.sum(dimensions[:(i+1)]) * rank)] = B_i

        elif i == 0:
            B_i = BT[len_dim - 2]
            JT[:dimensions[0] * rank] = B_i.reshape((-1,))
        else:

            small_permute = np.array([len_dim - 1])
            small_permute = np.append(small_permute, np.arange(len_dim - 1))
            small_dim = np.array(dimensions[len_dim - 1])
            small_dim = np.append(small_dim, dimensions[:len_dim - 1])

            B_i = None
            for r in range(rank):
                small_permute[0] = len_dim - 1
                prom = np.transpose(tensor, small_permute)
                for j in range(len_dim - 1, 0, -1):
                    prom = prom.reshape(-1, small_dim[j]) @ factors[j - 1][:, r]
                    prom = prom.reshape(small_dim[:j])
                    small_permute[0] -= 1
                if B_i is None:
                    B_i = prom
                else:
                    B_i = np.vstack((B_i, prom))
            if rank != 1:
                JT[-dimensions[-1] * rank :] = np.transpose(B_i, (1, 0)).reshape((-1,))
            else:
                JT[-dimensions[-1] * rank :] = B_i

    return JT

In [102]:
def Levenberg(tensor, rank, seed=42, tol = 10**(-8), n_iteration = 100):

    dimensions = tensor.shape
    len_dim = len(dimensions)

    jacob_size = int(np.sum(dimensions) * rank)

    rng = np.random.RandomState(seed)
    factors = np.array([rng.rand(dimensions[i], rank) for i in range(len_dim)], dtype=object)

    eps = 1.0
    it = 0

    pos = [0]
    for i in range(len_dim - 1):
        pos.append(pos[i] + dimensions[i] * rank)

    now_lambda = 0

    while eps > tol and it < n_iteration:

        layers = count_layers(len_dim, factors)
        W = count_W(len_dim, layers)
        JJ_d = count_JJ_d(len_dim, W, pos, jacob_size, dimensions, rank, factors)
        JA = calculate_JA(len_dim, factors, rank, dimensions, W, jacob_size)
        JT = calculate_JT(len_dim, factors, rank, dimensions, jacob_size, tensor)

        b = JA.reshape((-1,)) - JT.reshape((-1,))

        D_s = np.sqrt(np.diag(JJ_d))
        D_s_i = 1 / D_s


        b = D_s_i * b
        B = D_s_i.reshape((-1, 1)) * JJ_d * D_s_i
        T, U = lg.hessenberg(B, calc_q=True)

        b = U.T @ b

        mas = range(1, 3)

        min_lambda = None
        min_er = 0

        for alpha in mas:
            new_T = T + alpha * np.eye(jacob_size)
            y = lg.solve(new_T, b, assume_a='her')
            delta = U @ y
            delta = D_s_i * delta

            new_factors = []
            for i in range(len_dim):
                position = int(np.sum(dimensions[:i]) * rank)
                factor = np.zeros((dimensions[i], rank))
                for j in range(dimensions[i]):
                    factor[j, :] = factors[i][j, :] - (delta[position + j * rank:position + (j+1)*rank]).reshape(-1,)
                new_factors.append(factor)

            sweep_l = np.ones(rank).reshape(1, -1)
            for j in range(len_dim):
                sweep_l = lg.khatri_rao(sweep_l, new_factors[j])

            er = np.linalg.norm(np.sum(sweep_l, axis=1) - tensor.reshape(-1,)) / np.linalg.norm(tensor.reshape(-1,))
            if not min_lambda or er < min_er:
                min_lambda = alpha
                min_er = er

        now_lambda = min_lambda
        eps = min_er

        new_T = T + now_lambda * np.eye(jacob_size)
        y = lg.solve(new_T, b, assume_a='her')
        delta = U @ y
        delta = D_s_i * delta
        for i in range(len_dim):
            position = int(np.sum(dimensions[:i]) * rank)
            factor = np.array((dimensions[i], rank))
            for j in range(dimensions[i]):
                factors[i][j, :] -= (delta[position + j * rank:position + (j+1)*rank]).reshape(-1,)
        it += 1
        print(f'it={it}, min_er={min_er}')

    return factors, it

In [108]:
sizes = np.array((10, 20, 30))
T = np.zeros(sizes)
for i in range(sizes[0]):
    for j in range(sizes[1]):
        for k in range(sizes[2]):
            T[i, j, k] = 1 / (i + j + k + 1)

In [109]:
rank = 1
factors, it = Levenberg(T, rank)
print(it)

it=1, min_er=1.3193995158187835
it=2, min_er=0.7512901383399122
it=3, min_er=0.5076241696751754
it=4, min_er=0.35685918067053574
it=5, min_er=0.2938544139690723
it=6, min_er=0.27378320113100457
it=7, min_er=0.26737898751555794
it=8, min_er=0.2651930645119094
it=9, min_er=0.26439944023184164
it=10, min_er=0.26409798731916667
it=11, min_er=0.2639798861780716
it=12, min_er=0.2639326636247447
it=13, min_er=0.2639135326788189
it=14, min_er=0.263905718054865
it=15, min_er=0.2639025095197475
it=16, min_er=0.2639011879905008
it=17, min_er=0.26390064262668556
it=18, min_er=0.26390041730234437
it=19, min_er=0.26390032413969733
it=20, min_er=0.2639002856038196
it=21, min_er=0.26390026965957175
it=22, min_er=0.2639002630615625
it=23, min_er=0.2639002603309225
it=24, min_er=0.26390025920075755
it=25, min_er=0.2639002587329846
it=26, min_er=0.2639002585393702
it=27, min_er=0.2639002584592307
it=28, min_er=0.2639002584260597
it=29, min_er=0.26390025841232967
it=30, min_er=0.2639002584066465
it=31, mi

In [110]:
len_dim = len(sizes)
sweep_l = np.ones(rank).reshape(1, -1)

for j in range(len_dim):
    sweep_l = lg.khatri_rao(sweep_l, factors[j])

print(np.linalg.norm(np.sum(sweep_l, axis=1) - T.reshape(-1,)) / np.linalg.norm(T.reshape(-1,)))

0.26390025840263276
